In [2]:
import polars as pl
import pandas as pd
import duckdb as db

In [25]:
df=pl.read_parquet('../../data/processed/train_enhanced.parquet')
df=df.with_columns(pl.col("Discount").str.replace("Yes", "1").str.replace("No", "0").cast(pl.Int8))
df.head()

ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Order,Sales,Day_of_Week,Month,Year,Quarter,Week,Is_Weekend,log_Sales,log_Order,sqrt_Sales,sqrt_Order
str,i64,str,str,str,datetime[ns],i64,i8,i64,f64,str,str,i32,i32,u32,i64,f64,f64,f64,f64
"""T1000001""",1,"""S1""","""L3""","""R1""",2018-01-01 00:00:00,1,1,9,7011.84,"""Monday""","""January""",2018,1,1,0,8.855498,2.302585,83.73673,3.0
"""T1000002""",253,"""S4""","""L2""","""R1""",2018-01-01 00:00:00,1,1,60,51789.12,"""Monday""","""January""",2018,1,1,0,10.854955,4.110874,227.57223,7.745967
"""T1000003""",252,"""S3""","""L2""","""R1""",2018-01-01 00:00:00,1,1,42,36868.2,"""Monday""","""January""",2018,1,1,0,10.515132,3.7612,192.010937,6.480741
"""T1000004""",251,"""S2""","""L3""","""R1""",2018-01-01 00:00:00,1,1,23,19715.16,"""Monday""","""January""",2018,1,1,0,9.889194,3.178054,140.410683,4.795832
"""T1000005""",250,"""S2""","""L3""","""R4""",2018-01-01 00:00:00,1,1,62,45614.52,"""Monday""","""January""",2018,1,1,0,10.728003,4.143135,213.57556,7.874008


In [10]:
holidays = db.sql("""
    select distinct Date, Holiday from df
 """).to_df()
holidays.head()

,Date,Holiday
0,2019-01-23,0
1,2019-01-25,0
2,2019-02-09,0
3,2019-02-21,0
4,2019-02-25,0


In [11]:
holidays.to_csv('../../data/processed/holidays.csv', index=False)

In [21]:
db.sql("""
    select distinct Store_Type,Date, Discount from df order by  Date
""")

┌────────────┬─────────────────────┬──────────┐
│ Store_Type │        Date         │ Discount │
│  varchar   │    timestamp_ns     │ varchar  │
├────────────┼─────────────────────┼──────────┤
│ S1         │ 2018-01-01 00:00:00 │ Yes      │
│ S3         │ 2018-01-01 00:00:00 │ Yes      │
│ S2         │ 2018-01-01 00:00:00 │ Yes      │
│ S4         │ 2018-01-01 00:00:00 │ Yes      │
│ S2         │ 2018-01-02 00:00:00 │ Yes      │
│ S3         │ 2018-01-02 00:00:00 │ Yes      │
│ S4         │ 2018-01-02 00:00:00 │ Yes      │
│ S1         │ 2018-01-02 00:00:00 │ Yes      │
│ S2         │ 2018-01-03 00:00:00 │ Yes      │
│ S1         │ 2018-01-03 00:00:00 │ Yes      │
│ ·          │          ·          │ ·        │
│ ·          │          ·          │ ·        │
│ ·          │          ·          │ ·        │
│ S1         │ 2019-05-30 00:00:00 │ No       │
│ S1         │ 2019-05-30 00:00:00 │ Yes      │
│ S1         │ 2019-05-31 00:00:00 │ Yes      │
│ S4         │ 2019-05-31 00:00:00 │ Yes

In [ ]:
db.sql("""
    select distinct Store_id,Date, Discount from df where Store_id = 1 
""")

In [34]:
train_store_type_agg =db.sql("""
select 
        Date
        , Store_Type
        , cast(sum(Sales) as int) as Total_Sales
        , cast(avg(Sales) as int) as Avg_Sales
        , sum("Order") as Total_Orders
        , cast(avg("Order") as int) as Avg_Orders
        , count(distinct Store_id) as Num_Stores
        , max(Holiday) as Holiday
        , sum(Discount) as Total_Discounts
    from df
        group by 
       Date
       , Store_Type
    order by Date
""").pl()
train_store_type_agg.head()

Date,Store_Type,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
datetime[ns],str,i32,i32,"decimal[38,0]",i32,i64,i64,"decimal[38,0]"
2018-01-01 00:00:00,"""S4""",3728611,41895,4778,54,89,1,89
2018-01-01 00:00:00,"""S1""",7292028,42396,9410,55,172,1,172
2018-01-01 00:00:00,"""S3""",1944327,40507,2439,51,48,1,48
2018-01-01 00:00:00,"""S2""",2380519,42509,3039,54,56,1,56
2018-01-02 00:00:00,"""S2""",2023833,36140,2428,43,56,0,56


In [35]:
train_store_type_agg.write_parquet('../../data/processed/train_store_type_agg.parquet')

In [ ]:
train_region_code_agg =db.sql("""
    select 
        Date
        , Region_Code
        , cast(sum(Sales) as int) as Total_Sales
        , cast(avg(Sales) as int) as Avg_Sales
        , sum("Order") as Total_Orders
        , cast(avg("Order") as int) as Avg_Orders
        , count(distinct Store_id) as Num_Stores
        , max(Holiday) as Holiday
        , sum(Discount) as Total_Discounts
    from df
        group by 
       Date
       , Region_Code
    order by Date
""").pl()
train_region_code_agg.head()

Date,Region_Code,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
datetime[ns],str,i32,i32,"decimal[38,0]",i32,i64,i64,"decimal[38,0]"
2018-01-01 00:00:00,"""R4""",2286812,45736,2914,58,50,1,50
2018-01-01 00:00:00,"""R2""",4436859,42256,5644,54,105,1,105
2018-01-01 00:00:00,"""R1""",5094374,41084,6509,52,124,1,124
2018-01-01 00:00:00,"""R3""",3527439,41017,4599,53,86,1,86
2018-01-02 00:00:00,"""R3""",4622676,53752,5656,66,86,0,86


In [37]:
train_region_code_agg.write_parquet('../../data/processed/train_region_code_agg.parquet')

In [ ]:
train_location_type_agg =db.sql("""
    select 
        Date
        , Location_Type
        , cast(sum(Sales) as int) as Total_Sales
        , cast(avg(Sales) as int) as Avg_Sales
        , sum("Order") as Total_Orders
        , cast(avg("Order") as int) as Avg_Orders
        , count(distinct Store_id) as Num_Stores
        , max(Holiday) as Holiday
        , sum(Discount) as Total_Discounts
    from df
        group by 
       Date
       , Location_Type
    order by Date
""").pl()
train_location_type_agg.head()

Date,Location_Type,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
datetime[ns],str,i32,i32,"decimal[38,0]",i32,i64,i64,"decimal[38,0]"
2018-01-01 00:00:00,"""L5""",1170560,43354,1444,53,27,1,27
2018-01-01 00:00:00,"""L3""",2460245,42418,3175,55,58,1,58
2018-01-01 00:00:00,"""L2""",4117915,43808,5295,56,94,1,94
2018-01-01 00:00:00,"""L1""",6754876,40939,8662,52,165,1,165
2018-01-01 00:00:00,"""L4""",841889,40090,1090,52,21,1,21


In [39]:
train_location_type_agg.write_parquet('../../data/processed/train_location_type_agg.parquet')

In [ ]:
train_agg =db.sql("""
    select 
        Date
        , cast(sum(Sales) as int) as Total_Sales
        , cast(avg(Sales) as int) as Avg_Sales
        , sum("Order") as Total_Orders
        , cast(avg("Order") as int) as Avg_Orders
        , count(distinct Store_id) as Num_Stores
        , max(Holiday) as Holiday
        , sum(Discount) as Total_Discounts
    from df
        group by 
       Date
    order by Date
""").pl()
train_location_type_agg

Date,Location_Type,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
datetime[ns],str,i32,i32,"decimal[38,0]",i32,i64,i64,"decimal[38,0]"
2018-01-01 00:00:00,"""L5""",1170560,43354,1444,53,27,1,27
2018-01-01 00:00:00,"""L3""",2460245,42418,3175,55,58,1,58
2018-01-01 00:00:00,"""L2""",4117915,43808,5295,56,94,1,94
2018-01-01 00:00:00,"""L1""",6754876,40939,8662,52,165,1,165
2018-01-01 00:00:00,"""L4""",841889,40090,1090,52,21,1,21
…,…,…,…,…,…,…,…,…
2019-05-31 00:00:00,"""L5""",642061,23780,1009,37,27,1,1
2019-05-31 00:00:00,"""L3""",1877677,32374,2883,50,58,1,5
2019-05-31 00:00:00,"""L1""",6926767,41980,10286,62,165,1,13
